In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, callbacks
import matplotlib.pyplot as plt

In [ ]:
# The train, val and test sets are present in this drive link
# Please download the files and upload them in the content folder first

# https://drive.google.com/drive/folders/12GqRiyFHPUauuY0IIMlycVPRv_DuG5FI?usp=sharing

In [2]:
df1 = pd.read_csv('/content/drive/MyDrive/Datasets/Sentiment Analysis/train.txt', sep = ';', names = ['Text', 'Mood'])
df2 = pd.read_csv('/content/drive/MyDrive/Datasets/Sentiment Analysis/val.txt', sep = ';', names = ['Text', 'Mood'])
df3 = pd.read_csv('/content/drive/MyDrive/Datasets/Sentiment Analysis/test.txt', sep = ';', names = ['Text', 'Mood'])

df = pd.concat([df1, df2, df3])
df = df.iloc[:, 0].tolist()

In [3]:
# Creating the vocabulary from the training set

def create_vocab(df, r):
  vocab = {}
  c = 0

  for i in df:
    if r==1:
      i = i.split()
    for k in i:

      if k not in vocab.keys():
        vocab[k] = c
        c = c + 1

  return vocab

In [4]:
def divide(n, vocab, df, r):
  X = []
  y = []
  for i in df:
    if r==1:
      i = i.split()

    if len(i)>=n:
      for j in range(len(i)-n+1):
        x = []
        for k in range(n-1):
          x.append(vocab[i[j+k]])
        y.append([vocab[i[j+n-1]]])
        X.append(x)

  return X, y

In [7]:
r = int(input("0 for character; 1 for word: "))
n = int(input("Enter (number of input)+1 characters/words: "))
v = create_vocab(df, r)
X, y = divide(n, v, df, r)

0 for character; 1 for word: 0
Enter (number of input)+1 characters/words: 101


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

X_train = np.array(X_train[0:100000])
X_test = np.array(X_test)
y_train = np.array(y_train[0:100000])
y_test = np.array(y_test)

In [ ]:
# This is required if we do not use the embedding layer

# X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1])).astype('float64')
# X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1])).astype('float64')

In [ ]:
# Training the actual model

model = models.Sequential([
        layers.Embedding(len(v), 30),
        # layers.LSTM(128, return_sequences=True),
        layers.LSTM(128, return_sequences=True),
        layers.LSTM(100),
        # layers.Dense(100, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(len(v), activation='sigmoid')
])

model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer='adam')
ES = callbacks.EarlyStopping(patience=3, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.1, epochs=50, callbacks=[ES])

Epoch 1/50
2813/2813 [==============================] - 510s 180ms/step - loss: 2.3545 - accuracy: 0.3093 - val_loss: 2.0347 - val_accuracy: 0.3933
Epoch 2/50
2813/2813 [==============================] - 506s 180ms/step - loss: 1.9393 - accuracy: 0.4148 - val_loss: 1.8228 - val_accuracy: 0.4469
Epoch 3/50
2813/2813 [==============================] - 505s 179ms/step - loss: 1.7711 - accuracy: 0.4572 - val_loss: 1.7162 - val_accuracy: 0.4716
Epoch 4/50
2813/2813 [==============================] - 504s 179ms/step - loss: 1.6635 - accuracy: 0.4879 - val_loss: 1.6440 - val_accuracy: 0.4990
Epoch 5/50
2813/2813 [==============================] - 508s 180ms/step - loss: 1.5825 - accuracy: 0.5116 - val_loss: 1.6105 - val_accuracy: 0.5096
Epoch 6/50
2813/2813 [==============================] - 517s 184ms/step - loss: 1.5211 - accuracy: 0.5268 - val_loss: 1.5801 - val_accuracy: 0.5168
Epoch 7/50
2813/2813 [==============================] - 510s 181ms/step - loss: 1.4678 - accuracy: 0.5420 - val_

In [ ]:
y_pred = model.predict(X_test)
y_p = []

for i in y_pred:
  m = list(i).index(max(i))
  y_p.append(m)
y_test = np.reshape(y_test, (-1))

n = (y_p==y_test).sum()
acc = n/len(y_test)
print("Accuracy: ", acc)

Accuracy:  0.5369556867828509


In [ ]:
def custom(n, vocab, inp):
  x = []
  l = 1

  if len(inp)==n:
    for i in inp:
      x.append(vocab[i])

  else:
    l = 0
    print("Current length: ", len(inp))
    print("Enter a text of len: ", n)

  return [x], l

In [ ]:
t = input("Enter the text: ")
t_, j = custom(100, v, t)

if j==1:
  x = int(input("Enter the number of char to be predicted: "))
  print("The complete output is:\n")
  print(t, end='')
  for _ in range(x):
    pred = model.predict(t_)
    y = list(v.keys())
    z = list(v.values())
    m = list(pred[0]).index(max(pred[0]))
    val = y[z.index(m)]
    print(val, end='')
    t_[0].pop(0)
    t_[0].append(m)

Enter the text: i feel that i m so pathetic and downright dumb to let people in let them toy with my feelings and th
Enter the number of char to be predicted: 40
The complete output is:

i feel that i m so pathetic and downright dumb to let people in let them toy with my feelings and the struggled to the struggled to be a bal